In [1]:
import pandas as pd
import numpy as np

# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import KBinsDiscretizer

# from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.metrics import roc_auc_score

from catboost import CatBoostClassifier

c:\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\python37\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\python37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

**Train - Test Distribution**

<img src="./assets/train-test-dist.png" width="100%">

In [3]:
# 이상치가 있는 feature 파악

train_outlier = []
features = df_train.columns

for col in features[1:-1]:
    q1_train, q3_train = df_train[col].quantile(.25), df_train[col].quantile(.75)
    iqr_train = q3_train - q1_train
    col_now_train = df_train[col]
    if col_now_train[(q1_train-iqr_train*1.5 > df_train[col]) | (q3_train+iqr_train*1.5 < df_train[col])].count() > 0:
        train_outlier.append(col)

In [4]:
# 결측치를 채우기 전, 결측치 개수 자체를 특징으로 생성한 다음 결측치를 채운다.


df_train_without_nan = df_train.copy()
df_test_without_nan = df_test.copy()

# 결측치 개수 자체를 새로운 특징으로
df_train_without_nan['missing'] = df_train[features[1:-1]].isna().sum(axis=1)
df_test_without_nan['missing'] = df_test[features[1:-1]].isna().sum(axis=1)


# 이상치가 있는 feature 는 median으로,
# 이상치가 없는 feature 는 mean으로
for idx in range(1, 119):
    if f'f{idx}' in train_outlier:
        df_train_without_nan[f'f{idx}'] = df_train_without_nan[f'f{idx}'].fillna(df_train_without_nan[f'f{idx}'].median())
        df_test_without_nan[f'f{idx}'] = df_test_without_nan[f'f{idx}'].fillna(df_test_without_nan[f'f{idx}'].median())
    else:
        df_train_without_nan[f'f{idx}'] = df_train_without_nan[f'f{idx}'].fillna(df_train_without_nan[f'f{idx}'].mean())
        df_test_without_nan[f'f{idx}'] = df_test_without_nan[f'f{idx}'].fillna(df_test_without_nan[f'f{idx}'].mean())
        
        
# 정규화하기 전 분포 정보 자체를 특징으로
df_train_without_nan['std'] = df_train[features[1:-1]].std(axis=1)
df_test_without_nan['std'] = df_test[features[1:-1]].std(axis=1)

In [5]:
# 왜도 skewness 검사 |val| < 1.5

# strange = dict()
# for col in features[1:-1]:
#     col_skew = df_train_without_nan[col].skew()
#     if abs(col_skew) >= 1.5:
#         strange[col] = [col_skew, 0]

# # for col, col_skew in strange.items():
# #     print(f'{col:>4} :  {col_skew:>.4f}')



# # 직접 선별
# skew_for_log = ['f3', 'f4', 'f10', 'f21', 'f26', 'f32', 'f33', 'f38', 'f51', 'f53',
#                 'f64', 'f69', 'f74', 'f83', 'f92', 'f96', 'f99', 'f103', 'f104', 'f112',
#                 'f114', 'f115']
# skew_for_sqrt = ['f36', 'f62', 'f73', 'f116']


skew_for_log = ['f3', 'f4', 'f10', 'f21', 'f26', 'f32', 'f33', 'f38', 'f51', 'f53',
                'f64', 'f69', 'f74', 'f83', 'f92', 'f96', 'f99', 'f103', 'f104', 'f112',
                'f114', 'f115']
for col in skew_for_log:
    if min(df_train_without_nan[col]) < 0:
        df_train_without_nan[col] = np.log1p(df_train_without_nan[col] - min(df_train_without_nan[col]))
        df_test_without_nan[col] = np.log1p(df_test_without_nan[col] - min(df_test_without_nan[col]))
    else:
        df_train_without_nan[col] = np.log1p(df_train_without_nan[col])
        df_test_without_nan[col] = np.log1p(df_test_without_nan[col])

In [6]:
# 치우친 특성 => np.log1p(x - min(x))

# 양수 데이터만 있으면 그냥 np.log1p(x)
# 음수 데이터도 있으면 np.log1p(x - min(x)) => 데이터를 양수로 평행이동 뒤 정규화
# for col, _ in strange.items():
#     if min(df_train_without_nan[col]) >= 0:
#         df_train_without_nan[col] = np.log1p(df_train_without_nan[col])
#     else:
#         df_train_without_nan[col] = np.log1p(df_train_without_nan[col] - min(df_train_without_nan[col]))
        
#     if min(df_test_without_nan[col]) >= 0:
#         df_test_without_nan[col] = np.log1p(df_test_without_nan[col])
#     else:
#         df_test_without_nan[col] = np.log1p(df_test_without_nan[col] - min(df_test_without_nan[col]))

In [7]:
features = list(features[1:-1]) + ['missing', 'std'] # 새로 추가한 특성도 정규화가 필요할까?

# scaler = StandardScaler()
scaler = RobustScaler()
# scaler = MinMaxScaler()

# id, claim 제외
df_train_without_nan = scaler.fit_transform(df_train_without_nan[features])
df_test_without_nan = scaler.transform(df_test_without_nan[features])

df_train_without_nan = pd.DataFrame(df_train_without_nan, columns=features)
df_test_without_nan = pd.DataFrame(df_test_without_nan, columns=features)

In [8]:
# KBinsDiscretizer 이산화 전략
n_bins=5
kb = KBinsDiscretizer(n_bins=n_bins, strategy='uniform', encode='onehot-dense')

df_train_without_nan_kb = kb.fit_transform(df_train_without_nan)
df_test_without_nan_kb = kb.transform(df_test_without_nan)

In [9]:
# 구간마다 기울기 다르게 변형

df_train_tmp = np.repeat(df_train_without_nan.values, n_bins, axis=1) # 가로로 복제 (교차 복제는 np.tile)
df_train_without_nan = np.hstack([df_train_without_nan_kb, df_train_tmp * df_train_without_nan_kb])
del df_train_tmp
del df_train_without_nan_kb
df_train_without_nan = pd.DataFrame(df_train_without_nan)

df_test_tmp = np.repeat(df_test_without_nan.values, n_bins, axis=1)
df_test_without_nan = np.hstack([df_test_without_nan_kb, df_test_tmp * df_test_without_nan_kb])
del df_test_tmp
del df_test_without_nan_kb
df_test_without_nan = pd.DataFrame(df_test_without_nan)

In [11]:
%%time

# SKF = StratifiedKFold(n_splits=5)
SKF = KFold(n_splits=5)

cbc_oof = np.zeros(df_train_without_nan.shape[0])
cbc_pred = np.zeros(df_test_without_nan.shape[0])

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

for fold, (tr_idx, vld_idx) in enumerate(SKF.split(df_train_without_nan, df_train['claim'])):
#     x_tr = df_train_without_nan[features].iloc[tr_idx]
    x_tr = df_train_without_nan.iloc[tr_idx]
    y_tr = df_train['claim'].iloc[tr_idx]
    
#     x_vld = df_train_without_nan[features].iloc[vld_idx]
    x_vld = df_train_without_nan.iloc[vld_idx]
    y_vld = df_train['claim'].iloc[vld_idx]
    
    CBC = CatBoostClassifier(learning_rate=0.05,
                              max_depth=8,
                              rsm=0.8822146506051032,
                              loss_function='Logloss',
                              eval_metric='AUC',
                              l2_leaf_reg=3.168429538076496,
                              random_strength=5.6809865305797045,
                              min_data_in_leaf=44,
                              num_boost_round=1000,
                              early_stopping_rounds=200,
                              random_seed=0)
    
    CBC.fit(x_tr, y_tr,
            eval_set=[(x_vld, y_vld)],
            early_stopping_rounds=200,
            verbose=1)

    cbc_oof[vld_idx] = CBC.predict_proba(x_vld)[:, -1]
    cbc_pred += CBC.predict_proba(df_test_without_nan)[:, -1] / 5
    
    auc = roc_auc_score(y_vld, cbc_oof[vld_idx])
    print(f'fold {fold} - CBC auc: {auc:.6f}\n')

#     이산화 전략 사용 시 Feature가 바뀌므로 feature_importances_ 사용 어려움?
#     feat_importances = pd.DataFrame({'feature_importance': CBC.feature_importances_}, index=features)
#     feat_importances = feat_importances.sort_values(by=['feature_importance'], ascending=False)
#     display(feat_importances)

0:	test: 0.7778073	best: 0.7778073 (0)	total: 888ms	remaining: 14m 47s
1:	test: 0.7783042	best: 0.7783042 (1)	total: 1.42s	remaining: 11m 49s
2:	test: 0.7907602	best: 0.7907602 (2)	total: 2.74s	remaining: 15m 11s
3:	test: 0.7913634	best: 0.7913634 (3)	total: 3.42s	remaining: 14m 10s
4:	test: 0.7914236	best: 0.7914236 (4)	total: 4s	remaining: 13m 16s
5:	test: 0.7914020	best: 0.7914236 (4)	total: 4.47s	remaining: 12m 21s
6:	test: 0.7913877	best: 0.7914236 (4)	total: 5.04s	remaining: 11m 55s
7:	test: 0.7916560	best: 0.7916560 (7)	total: 5.56s	remaining: 11m 29s
8:	test: 0.7916743	best: 0.7916743 (8)	total: 6.1s	remaining: 11m 11s
9:	test: 0.7917753	best: 0.7917753 (9)	total: 6.67s	remaining: 11m
10:	test: 0.8005983	best: 0.8005983 (10)	total: 7.16s	remaining: 10m 43s
11:	test: 0.8006851	best: 0.8006851 (11)	total: 7.64s	remaining: 10m 29s
12:	test: 0.8006623	best: 0.8006851 (11)	total: 8.22s	remaining: 10m 24s
13:	test: 0.8006823	best: 0.8006851 (11)	total: 8.71s	remaining: 10m 13s
14:	te

115:	test: 0.8079286	best: 0.8079286 (115)	total: 1m 2s	remaining: 7m 56s
116:	test: 0.8079700	best: 0.8079700 (116)	total: 1m 3s	remaining: 7m 55s
117:	test: 0.8081267	best: 0.8081267 (117)	total: 1m 3s	remaining: 7m 54s
118:	test: 0.8081661	best: 0.8081661 (118)	total: 1m 4s	remaining: 7m 54s
119:	test: 0.8083284	best: 0.8083284 (119)	total: 1m 4s	remaining: 7m 54s
120:	test: 0.8083198	best: 0.8083284 (119)	total: 1m 5s	remaining: 7m 53s
121:	test: 0.8083486	best: 0.8083486 (121)	total: 1m 5s	remaining: 7m 52s
122:	test: 0.8084383	best: 0.8084383 (122)	total: 1m 6s	remaining: 7m 51s
123:	test: 0.8084294	best: 0.8084383 (122)	total: 1m 6s	remaining: 7m 50s
124:	test: 0.8085017	best: 0.8085017 (124)	total: 1m 7s	remaining: 7m 50s
125:	test: 0.8085491	best: 0.8085491 (125)	total: 1m 7s	remaining: 7m 49s
126:	test: 0.8085950	best: 0.8085950 (126)	total: 1m 8s	remaining: 7m 48s
127:	test: 0.8085747	best: 0.8085950 (126)	total: 1m 8s	remaining: 7m 49s
128:	test: 0.8085692	best: 0.8085950 (

225:	test: 0.8106157	best: 0.8106203 (224)	total: 1m 59s	remaining: 6m 49s
226:	test: 0.8106187	best: 0.8106203 (224)	total: 2m	remaining: 6m 49s
227:	test: 0.8106476	best: 0.8106476 (227)	total: 2m	remaining: 6m 48s
228:	test: 0.8106683	best: 0.8106683 (228)	total: 2m 1s	remaining: 6m 47s
229:	test: 0.8106614	best: 0.8106683 (228)	total: 2m 1s	remaining: 6m 47s
230:	test: 0.8106688	best: 0.8106688 (230)	total: 2m 2s	remaining: 6m 46s
231:	test: 0.8106946	best: 0.8106946 (231)	total: 2m 2s	remaining: 6m 45s
232:	test: 0.8106933	best: 0.8106946 (231)	total: 2m 3s	remaining: 6m 45s
233:	test: 0.8107348	best: 0.8107348 (233)	total: 2m 3s	remaining: 6m 44s
234:	test: 0.8107616	best: 0.8107616 (234)	total: 2m 4s	remaining: 6m 44s
235:	test: 0.8107698	best: 0.8107698 (235)	total: 2m 4s	remaining: 6m 43s
236:	test: 0.8108244	best: 0.8108244 (236)	total: 2m 5s	remaining: 6m 42s
237:	test: 0.8108332	best: 0.8108332 (237)	total: 2m 5s	remaining: 6m 42s
238:	test: 0.8108712	best: 0.8108712 (238)	

335:	test: 0.8125796	best: 0.8125796 (335)	total: 2m 57s	remaining: 5m 49s
336:	test: 0.8125602	best: 0.8125796 (335)	total: 2m 57s	remaining: 5m 49s
337:	test: 0.8126221	best: 0.8126221 (337)	total: 2m 58s	remaining: 5m 48s
338:	test: 0.8126290	best: 0.8126290 (338)	total: 2m 58s	remaining: 5m 48s
339:	test: 0.8126499	best: 0.8126499 (339)	total: 2m 59s	remaining: 5m 47s
340:	test: 0.8126609	best: 0.8126609 (340)	total: 2m 59s	remaining: 5m 47s
341:	test: 0.8127062	best: 0.8127062 (341)	total: 3m	remaining: 5m 46s
342:	test: 0.8126942	best: 0.8127062 (341)	total: 3m	remaining: 5m 46s
343:	test: 0.8126976	best: 0.8127062 (341)	total: 3m 1s	remaining: 5m 45s
344:	test: 0.8127084	best: 0.8127084 (344)	total: 3m 1s	remaining: 5m 45s
345:	test: 0.8127133	best: 0.8127133 (345)	total: 3m 2s	remaining: 5m 44s
346:	test: 0.8127421	best: 0.8127421 (346)	total: 3m 2s	remaining: 5m 44s
347:	test: 0.8127922	best: 0.8127922 (347)	total: 3m 3s	remaining: 5m 43s
348:	test: 0.8127961	best: 0.8127961 (

445:	test: 0.8138407	best: 0.8138407 (445)	total: 3m 54s	remaining: 4m 51s
446:	test: 0.8138419	best: 0.8138419 (446)	total: 3m 55s	remaining: 4m 51s
447:	test: 0.8138633	best: 0.8138633 (447)	total: 3m 55s	remaining: 4m 50s
448:	test: 0.8138568	best: 0.8138633 (447)	total: 3m 56s	remaining: 4m 50s
449:	test: 0.8138608	best: 0.8138633 (447)	total: 3m 56s	remaining: 4m 49s
450:	test: 0.8138598	best: 0.8138633 (447)	total: 3m 57s	remaining: 4m 49s
451:	test: 0.8138586	best: 0.8138633 (447)	total: 3m 57s	remaining: 4m 48s
452:	test: 0.8138714	best: 0.8138714 (452)	total: 3m 58s	remaining: 4m 47s
453:	test: 0.8138790	best: 0.8138790 (453)	total: 3m 58s	remaining: 4m 47s
454:	test: 0.8139002	best: 0.8139002 (454)	total: 3m 59s	remaining: 4m 46s
455:	test: 0.8139111	best: 0.8139111 (455)	total: 4m	remaining: 4m 46s
456:	test: 0.8139159	best: 0.8139159 (456)	total: 4m	remaining: 4m 45s
457:	test: 0.8139182	best: 0.8139182 (457)	total: 4m 1s	remaining: 4m 45s
458:	test: 0.8139163	best: 0.81391

...

538:	test: 0.8127432	best: 0.8127513 (537)	total: 4m 46s	remaining: 4m 4s
539:	test: 0.8127421	best: 0.8127513 (537)	total: 4m 46s	remaining: 4m 4s
540:	test: 0.8127495	best: 0.8127513 (537)	total: 4m 47s	remaining: 4m 3s
541:	test: 0.8127652	best: 0.8127652 (541)	total: 4m 47s	remaining: 4m 3s
542:	test: 0.8127664	best: 0.8127664 (542)	total: 4m 48s	remaining: 4m 2s
543:	test: 0.8127628	best: 0.8127664 (542)	total: 4m 48s	remaining: 4m 2s
544:	test: 0.8127692	best: 0.8127692 (544)	total: 4m 49s	remaining: 4m 1s
545:	test: 0.8127736	best: 0.8127736 (545)	total: 4m 49s	remaining: 4m
546:	test: 0.8127723	best: 0.8127736 (545)	total: 4m 50s	remaining: 4m
547:	test: 0.8127813	best: 0.8127813 (547)	total: 4m 50s	remaining: 3m 59s
548:	test: 0.8127788	best: 0.8127813 (547)	total: 4m 51s	remaining: 3m 59s
549:	test: 0.8127916	best: 0.8127916 (549)	total: 4m 52s	remaining: 3m 58s
550:	test: 0.8127698	best: 0.8127916 (549)	total: 4m 52s	remaining: 3m 58s
551:	test: 0.8127830	best: 0.8127916 (54

648:	test: 0.8130394	best: 0.8130394 (648)	total: 5m 43s	remaining: 3m 5s
649:	test: 0.8130400	best: 0.8130400 (649)	total: 5m 44s	remaining: 3m 5s
650:	test: 0.8130471	best: 0.8130471 (650)	total: 5m 44s	remaining: 3m 4s
651:	test: 0.8130474	best: 0.8130474 (651)	total: 5m 45s	remaining: 3m 4s
652:	test: 0.8130481	best: 0.8130481 (652)	total: 5m 45s	remaining: 3m 3s
653:	test: 0.8130602	best: 0.8130602 (653)	total: 5m 46s	remaining: 3m 3s
654:	test: 0.8130645	best: 0.8130645 (654)	total: 5m 46s	remaining: 3m 2s
655:	test: 0.8130703	best: 0.8130703 (655)	total: 5m 47s	remaining: 3m 2s
656:	test: 0.8130746	best: 0.8130746 (656)	total: 5m 47s	remaining: 3m 1s
657:	test: 0.8130787	best: 0.8130787 (657)	total: 5m 48s	remaining: 3m 1s
658:	test: 0.8130981	best: 0.8130981 (658)	total: 5m 48s	remaining: 3m
659:	test: 0.8131137	best: 0.8131137 (659)	total: 5m 49s	remaining: 2m 59s
660:	test: 0.8130953	best: 0.8131137 (659)	total: 5m 49s	remaining: 2m 59s
661:	test: 0.8130880	best: 0.8131137 (6

758:	test: 0.8132800	best: 0.8132821 (757)	total: 6m 40s	remaining: 2m 7s
759:	test: 0.8132769	best: 0.8132821 (757)	total: 6m 41s	remaining: 2m 6s
760:	test: 0.8132829	best: 0.8132829 (760)	total: 6m 41s	remaining: 2m 6s
761:	test: 0.8132806	best: 0.8132829 (760)	total: 6m 42s	remaining: 2m 5s
762:	test: 0.8132819	best: 0.8132829 (760)	total: 6m 42s	remaining: 2m 5s
763:	test: 0.8132890	best: 0.8132890 (763)	total: 6m 43s	remaining: 2m 4s
764:	test: 0.8132900	best: 0.8132900 (764)	total: 6m 43s	remaining: 2m 4s
765:	test: 0.8132857	best: 0.8132900 (764)	total: 6m 44s	remaining: 2m 3s
766:	test: 0.8132948	best: 0.8132948 (766)	total: 6m 44s	remaining: 2m 2s
767:	test: 0.8133055	best: 0.8133055 (767)	total: 6m 45s	remaining: 2m 2s
768:	test: 0.8133054	best: 0.8133055 (767)	total: 6m 45s	remaining: 2m 1s
769:	test: 0.8133265	best: 0.8133265 (769)	total: 6m 46s	remaining: 2m 1s
770:	test: 0.8133376	best: 0.8133376 (770)	total: 6m 46s	remaining: 2m
771:	test: 0.8133466	best: 0.8133466 (771

868:	test: 0.8135842	best: 0.8135875 (867)	total: 7m 39s	remaining: 1m 9s
869:	test: 0.8135823	best: 0.8135875 (867)	total: 7m 39s	remaining: 1m 8s
870:	test: 0.8135792	best: 0.8135875 (867)	total: 7m 40s	remaining: 1m 8s
871:	test: 0.8135747	best: 0.8135875 (867)	total: 7m 40s	remaining: 1m 7s
872:	test: 0.8135850	best: 0.8135875 (867)	total: 7m 41s	remaining: 1m 7s
873:	test: 0.8135936	best: 0.8135936 (873)	total: 7m 42s	remaining: 1m 6s
874:	test: 0.8135959	best: 0.8135959 (874)	total: 7m 42s	remaining: 1m 6s
875:	test: 0.8135908	best: 0.8135959 (874)	total: 7m 43s	remaining: 1m 5s
876:	test: 0.8135941	best: 0.8135959 (874)	total: 7m 43s	remaining: 1m 5s
877:	test: 0.8135858	best: 0.8135959 (874)	total: 7m 44s	remaining: 1m 4s
878:	test: 0.8135817	best: 0.8135959 (874)	total: 7m 44s	remaining: 1m 3s
879:	test: 0.8135803	best: 0.8135959 (874)	total: 7m 45s	remaining: 1m 3s
880:	test: 0.8135787	best: 0.8135959 (874)	total: 7m 45s	remaining: 1m 2s
881:	test: 0.8135925	best: 0.8135959 (

980:	test: 0.8136852	best: 0.8136862 (979)	total: 8m 37s	remaining: 10s
981:	test: 0.8136851	best: 0.8136862 (979)	total: 8m 38s	remaining: 9.51s
982:	test: 0.8136846	best: 0.8136862 (979)	total: 8m 39s	remaining: 8.97s
983:	test: 0.8136846	best: 0.8136862 (979)	total: 8m 39s	remaining: 8.45s
984:	test: 0.8136785	best: 0.8136862 (979)	total: 8m 40s	remaining: 7.92s
985:	test: 0.8136785	best: 0.8136862 (979)	total: 8m 40s	remaining: 7.39s
986:	test: 0.8136751	best: 0.8136862 (979)	total: 8m 41s	remaining: 6.86s
987:	test: 0.8136680	best: 0.8136862 (979)	total: 8m 41s	remaining: 6.34s
988:	test: 0.8136771	best: 0.8136862 (979)	total: 8m 42s	remaining: 5.81s
989:	test: 0.8136823	best: 0.8136862 (979)	total: 8m 42s	remaining: 5.28s
990:	test: 0.8136736	best: 0.8136862 (979)	total: 8m 43s	remaining: 4.75s
991:	test: 0.8136705	best: 0.8136862 (979)	total: 8m 43s	remaining: 4.22s
992:	test: 0.8136749	best: 0.8136862 (979)	total: 8m 44s	remaining: 3.69s
993:	test: 0.8136733	best: 0.8136862 (97

In [12]:
target = df_train['claim']
print(f'oof cbc roc = {roc_auc_score(target, cbc_oof)}')

oof cbc roc = 0.8139626589428676


In [13]:
df_submission = pd.read_csv('./sample_solution.csv')
df_submission['claim'] = cbc_pred
df_submission.to_csv('./CatBoost-015.csv', index=False)